In [88]:
# encoding=utf-8
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow_addons.text import crf

class myModel(keras.Model):
    def __init__(self,time_steps,batch_size):
        super(myModel,self).__init__()
        self.time_steps = time_steps
        self.batch_size = batch_size

        self.fw_LSTM = layers.LSTM(units=100,activation='relu',return_sequences=True,go_backwards=False)
        self.bw_LSTM = layers.LSTM(units=100,activation='relu',return_sequences=True,go_backwards=True)
        self.BiLSTM = layers.Bidirectional(self.fw_LSTM,backward_layer=self.bw_LSTM,input_shape=(self.batch_size, self.time_steps)) #todo

    def call(self, inputs, training=None, mask=None):
        x = self.BiLSTM(inputs)
        return x

    def embedding_layer(self):
        pass # todo

batch_size = 2
time_steps = 5

x1 = np.array([[1,2,3,4,5],[3,1,5,2,1],[3,4,5,0,2],[5,3,4,0,1]],dtype='int32')
y1 = np.array([[0,0,1,2,3],
               [1,0,0,0,0],
               [1,2,3,0,0],
               [0,1,2,0,0]],dtype='int32')
lens = np.array([5,5,5,5])
model = myModel(batch_size,time_steps)




In [113]:
LSTM_dim = 50
tag_num = 4
fw_LSTM = layers.LSTM(units=LSTM_dim,return_sequences=True,go_backwards=False)
bw_LSTM = layers.LSTM(units=LSTM_dim,return_sequences=True,go_backwards=True)
# BiLSTM = layers.Bidirectional(fw_LSTM,backward_layer=bw_LSTM,input_shape=(4, 5)) #todo
BiLSTM = layers.Bidirectional(fw_LSTM,backward_layer=bw_LSTM) #todo

# x3 = tf.random.normal((2,5,1))
# x3
# x = BiLSTM(x)
if tf.test.is_gpu_available():
  with tf.device("/gpu:0"):
    train_loss = keras.metrics.Mean(name='train_loss')
    train_acc = keras.metrics.CategoricalAccuracy(name='train_acc')


    initializer = tf.keras.initializers.GlorotUniform()
    trans_p = tf.Variable(initializer([tag_num, tag_num]), trainable=True, name="transitions")

    w_init = tf.random_normal_initializer()
    w_proj = tf.Variable(w_init([100,tag_num]),trainable=True)
    b_init = tf.zeros_initializer()
    b_proj = tf.Variable(b_init([tag_num]),trainable=True)
    proj_layer = layers.Dense(tag_num)
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    for epoch in range(0,50):
        with tf.GradientTape(persistent=True) as tape:
            # x = tf.keras.layers.Embedding(6, 1, mask_zero=True)(x1)
            x = tf.expand_dims(x1,axis=-1)
            x = tf.cast(x,dtype='float')
            x = BiLSTM(x)
            # 全连接映射层
            pred = proj_layer(x)
            # pred = tf.matmul(x,w_proj) + b_proj
            # print(pred)

            # crf
            loss, trans_p = crf.crf_log_likelihood(inputs=pred,tag_indices=y1,transition_params=trans_p,sequence_lengths=lens)
            loss = tf.reduce_sum(loss)
            seq_tags, best_score = crf.crf_decode(potentials=pred,transition_params=trans_p, sequence_length=lens)
            # print(trans_p.shape)
        if epoch%10==0:
            print(epoch)
            print(seq_tags)
            # print(trans_p)
            # print(BiLSTM.trainable_variables[0][1])

        grads = tape.gradient(loss,[BiLSTM.trainable_variables,trans_p])
        grads2 = tape.gradient(loss,proj_layer.trainable_variables)
        # grads_crf = tape.gradient(loss,trans_p)
        grads_and_vars_clip0 = [[tf.clip_by_value(g, -5, 5), v] for g, v in zip(grads[0], BiLSTM.trainable_variables)]
        grads_and_vars_clip1 = [[tf.clip_by_value(g, -5, 5), v] for g, v in zip([grads[1]], [trans_p])]

        optimizer.apply_gradients(grads_and_vars_clip0)
        optimizer.apply_gradients(grads_and_vars_clip1)
        optimizer.apply_gradients(zip(grads2,proj_layer.trainable_variables))

    # train_loss.reset_states()
        train_acc.reset_states()
    # train_loss(loss)
        train_acc(y1,seq_tags)
        template = 'Epoch {}, train acc: {}'
        print (template.format(epoch+1,train_acc.result()*100))
    #                        train_loss.result()))




0
tf.Tensor(
[[2 0 1 0 1]
 [0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 1 0]], shape=(4, 5), dtype=int32)
Epoch 1, train acc: 0.0
Epoch 2, train acc: 0.0
Epoch 3, train acc: 0.0
Epoch 4, train acc: 0.0
Epoch 5, train acc: 0.0
Epoch 6, train acc: 0.0
Epoch 7, train acc: 0.0
Epoch 8, train acc: 0.0
Epoch 9, train acc: 0.0
Epoch 10, train acc: 0.0
10
tf.Tensor(
[[2 3 2 3 2]
 [2 3 2 3 2]
 [2 3 2 3 2]
 [2 3 2 3 2]], shape=(4, 5), dtype=int32)
Epoch 11, train acc: 0.0
Epoch 12, train acc: 0.0
Epoch 13, train acc: 0.0
Epoch 14, train acc: 0.0
Epoch 15, train acc: 0.0
Epoch 16, train acc: 25.0
Epoch 17, train acc: 25.0
Epoch 18, train acc: 25.0
Epoch 19, train acc: 25.0
Epoch 20, train acc: 25.0
20
tf.Tensor(
[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]], shape=(4, 5), dtype=int32)
Epoch 21, train acc: 25.0
Epoch 22, train acc: 25.0
Epoch 23, train acc: 25.0
Epoch 24, train acc: 25.0
Epoch 25, train acc: 25.0
Epoch 26, train acc: 25.0
Epoch 27, train acc: 25.0
Epoch 28, train acc: 25.0
Epoch 29, tr

In [ ]:
loss_object = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name='train_loss')
train_acc = keras.metrics.SparseTopKCategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

def train_step(input, labels):
    with tf.GradientTape() as tape:
        predict = model(input)
        loss = loss_object(predict, y1)

In [48]:
x=tf.constant(1.0)
w=tf.constant(2.0)
y=tf.Variable(1.0,trainable=True)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    z = w*y*y*y
    func1 = x*z
df_dx = tape.gradient(func1,x)
dz_dy = tape.gradient(func1,y)
grads = tape.gradient(func1,[x,y])
print(df_dx)
print(dz_dy)
print(grads)

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [1]:
import pickle
path = r'F:\zzd\毕业论文\论文代码\DataSets\2014人民日报\vocab_test.pkl'
with open(path, 'rb') as f:
    word2id = pickle.load(f)
print(type(word2id))
print(word2id['你'])

<class 'dict'>
635


In [33]:
import tensorflow as tf
import numpy as np
l = [[[1,2,3],[4,5,6]]]
p = tf.convert_to_tensor(l)
print(p)
s = np.array(p)
l = s.flatten()
lo = []
lo.append(l)
print(lo)
for i in lo:
    for l in i:
        print(l)

tf.Tensor(
[[[1 2 3]
  [4 5 6]]], shape=(1, 2, 3), dtype=int32)
[array([1, 2, 3, 4, 5, 6])]
1
2
3
4
5
6


In [6]:
l = 2.35625489154
print('number is {:.4f}'.format(l))

number is 2.3563


In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import time
import os
x = [[1,2,3,4,5]]
y = [1,1,0,0,1]

LSTM_dim = 100
tag_num = 2
# 模型所需的层定义
embedding = layers.Embedding(input_dim=10000, output_dim=2, mask_zero=True)
dense = layers.Dense(tag_num)
# fw_LSTM = layers.LSTM(units=LSTM_dim, return_sequences=True, go_backwards=False)
# bw_LSTM = layers.LSTM(units=LSTM_dim, return_sequences=True, go_backwards=True)
# BiLSTM = layers.Bidirectional(fw_LSTM, backward_layer=bw_LSTM)
BiLSTM = layers.Bidirectional(layers.LSTM(100,return_sequences=True))

x = tf.convert_to_tensor(x,dtype='int32')
z = embedding(x)
z = BiLSTM(z)
z = dense(z)
print(z)

E:\Tools\Anaconda\envs\TF2.1\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


tf.Tensor(
[[[ 6.1379164e-04  2.7616858e-05]
  [ 6.5122318e-04  4.2452704e-04]
  [ 4.4814346e-04  6.5117230e-04]
  [ 3.3737579e-04  2.4767499e-04]
  [ 8.7461900e-04 -1.9915166e-04]]], shape=(1, 5, 2), dtype=float32)


In [15]:
from tensorflow_addons.text import crf

vocab_size = 10000
embedding_dim = 5
model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    # layers.GlobalAveragePooling1D(),
    # layers.Dense(160, activation='relu'),
    # layers.Dense(1, activation='sigmoid')
    # layers.Dense(160, activation='relu'),
    layers.Bidirectional(layers.LSTM(2,return_sequences=True)),
    layers.Dense(2)
])
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.BinaryCrossentropy(),
             metrics=['accuracy'])
x = [[1,2,3,4,5],[5,4,3,2,1]]
y = [[1,1,1,1,0],[1,1,1,1,1]]
lens = [5,5]
x = tf.constant(x)
y = tf.constant(y)
# lens = tf.constant(lens)
# opt = keras.optimizers.Adam()
with tf.GradientTape(persistent=True) as tape:
    z = model.predict(x)
    loss,transparam = crf.crf_log_likelihood(z,y,lens)
    # loss = tf.reduce_mean(loss)
grads1 = tape.gradient(loss,model.variables)
# grads2 = tape.gradient(loss,transparam)
# opt.apply_gradients(zip(grads1,model.variables))
# opt.apply_gradients(zip(grads2,transparam))


ValueError: Tried to convert 'tensor' to a tensor and failed. Error: None values not supported.

In [37]:
x =tf.Variable(initial_value=2.)
y=tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    z = 6*x
    p = z*y**2
grads = tape.gradient(p,[x,y])
print(grads)

[<tf.Tensor: shape=(), dtype=float32, numpy=54.0>, <tf.Tensor: shape=(), dtype=float32, numpy=72.0>]


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow_addons.text import crf

tf.test.is_built_with_cuda()

E:\Tools\Anaconda\envs\TF2.1\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


True

In [ ]:
score = [[
    [1, 2, 3],
    [2, 1, 3],
    [1, 3, 2],
    [3, 2, 1],
    [1,1,1],

]]  # (batch_size, time_step, num_tabs)
transition = [
    [2, 1, 3],
    [1, 3, 2],
    [3, 2, 1]
]   # (num_tabs, num_tabs)
lengths = [3]   # (batch_size, time_step)

score_t         = tf.constant(score, dtype=tf.int64)
transition_t    = tf.constant(transition, dtype=tf.int64)
lengths_t       = tf.constant(lengths, dtype=tf.int64)

# r1 = crf.crf_decode(score1,m,ls)
r2 = crf.crf_decode(
    potentials=score_t,
    transition_params=transition_t,
    sequence_length=[4])
r3 = crf.viterbi_decode(np.array(score[0]),np.array(transition))
# r4 = crf.viterbi_decode(score2,m)
# print(r1)
print(r2)
print(r3)
# print(r4)
s = tf.required_space_to_batch_paddings

In [33]:
s = [[1,2,3],[4,5,6,7],[1,2,3,4,5,6,7,8,9]]
ls = [3,4,9]
print(ls +1)
p = keras.preprocessing.sequence.pad_sequences(s,padding='post',value=0,truncating='post')
p = keras.preprocessing.sequence.pad_sequences(padding='post',value=0,truncating='post')
print(p)

TypeError: can only concatenate list (not "int") to list